<a href="https://colab.research.google.com/github/WhitneyH1317/puma_permeability_calibration/blob/main/06_EcoScape_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare working environment

In [2]:
import os
import glob


In [3]:
# Install git-lfs and clone your repo
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

# Clone GitHub repo (replace YourUser and your-repo if you fork repo to your own account)
!git clone https://github.com/WhitneyH1317/puma_permeability_calibration.git
%cd puma_permeability_calibration

# Pull large files managed by Git LFS
!git lfs pull

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.14
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Need to get 8,917 kB of archives.
After this operation, 8,712 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.7.0 [8,917 kB]
Fetched 8,917 kB in 1s (8,796 kB/s)
debconf: unable to initialize frontend: Dialog
debc

In [4]:
!pip install ecoscape-connectivity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 110.8 MB/s eta 0:00:00


In [5]:
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv

In [6]:
# Gender choice.
GENDERS = "mal", "fem"
FULL_GENDERS = "male", "female"

# Define base repo directory and data version
REPO_DIR = "/content/puma_permeability_calibration"
DATA_VERSION = "2025-09-25"  # update to whichever data version you created in R


## Connectivity and Flow functions

In [10]:
def do_conn_and_flow(gender, full_gender):
    # Input layers and information.
    # Get all permeability raster files

    perm_file_pattern = os.path.join(REPO_DIR, f"rasters_for_collab/{DATA_VERSION}/perm_rasters_{full_gender}/permeability_*_{gender}*.tif")
    perm_file_incl = f"_{gender}_raw"
    perm_file_incl_ = perm_file_incl + "_"

    permeability_files = glob.glob(perm_file_pattern)
    # Parameters
    gap_crossing = 0 # no gap crossing.
    dispersal = 300 # pixel = 300m; how many pumas created how many pixels; how far they can go (this is length of raster)
    # make it too big and cannot compute the flow; Luca usually uses 100
    permeability_scaling = 10 / dispersal
    # Parameters that you should change only with care
    num_simulations = 1000 # Number of simulations to average (at least 1k); bump up if needed
    seed_density = 10 # increased from 4 just to see

    # Parameters for computation.
    tile_size = 1024
    tile_border = dispersal # should match dispersal

    # Define output directory
    output_dir = os.path.join(f"/content/output/flow_output_{full_gender}")

    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get all permeability raster files
    permeability_files = glob.glob(perm_file_pattern)

    # Loop through each permeability file
    for permeability_fn in permeability_files:
        # Extract personality type from filename
        file_only = os.path.basename(permeability_fn)  # this will just be 'permeability_high_fem_raw.tif'
        base_name = file_only.split("permeability_")[1].split(perm_file_incl)[0]  # yields 'high'

        # Check if filename contains year and scenario
        if perm_file_incl_ in permeability_fn:
            year_scenario = permeability_fn.split(perm_file_incl_)[1].replace(".tif", "")
        else:
            year_scenario = "current"

        # Define output filenames
        connectivity_fn = os.path.join(output_dir, f"connectivity_{base_name}_{year_scenario}.tif")
        flow_fn = os.path.join(output_dir, f"flow_{base_name}_{year_scenario}.tif")

        print(f"Processing: {base_name}, Scenario: {year_scenario}")

        # Run the function
        compute_connectivity(
            habitat_fn=None,
            permeability_fn=permeability_fn,
            permeability_scaling=permeability_scaling,
            connectivity_fn=connectivity_fn,
            flow_fn=flow_fn,
            gap_crossing=gap_crossing,
            dispersal=dispersal,
            num_simulations=num_simulations,
            seed_density=seed_density,
            tile_size=tile_size,
            border_size=tile_border,
            padding_size=tile_border,
            repopulation_only_in_habitat=False,
        )

        print(f"Saved to: {connectivity_fn}, {flow_fn}")


In [11]:
# Choose here what to run.
print("=== FEMALES ===")
do_conn_and_flow("fem", "female")

print("=== MALES ===")
do_conn_and_flow("mal", "male")


=== FEMALES ===
Processing: mix, Scenario: current
Saved to: /content/output/flow_output_female/connectivity_mix_current.tif, /content/output/flow_output_female/flow_mix_current.tif
Processing: high, Scenario: current
Saved to: /content/output/flow_output_female/connectivity_high_current.tif, /content/output/flow_output_female/flow_high_current.tif
Processing: low, Scenario: current
Saved to: /content/output/flow_output_female/connectivity_low_current.tif, /content/output/flow_output_female/flow_low_current.tif
=== MALES ===


# **Download results in a zip file**

In [16]:
from google.colab import files
import os

output_dir = "/content/output"
zip_path = "/content/output.zip"

# Zip the entire output directory recursively
!zip -r {zip_path} {output_dir}

# Download the zip file to local machine
files.download(zip_path)


  adding: content/output/ (stored 0%)
  adding: content/output/flow_output_female/ (stored 0%)
  adding: content/output/flow_output_female/flow_high_current.tif (deflated 10%)
  adding: content/output/flow_output_female/flow_low_current.tif (deflated 11%)
  adding: content/output/flow_output_female/flow_mix_current.tif (deflated 11%)
  adding: content/output/flow_output_female/connectivity_mix_current.tif (deflated 9%)
  adding: content/output/flow_output_female/connectivity_high_current.tif (deflated 9%)
  adding: content/output/flow_output_female/connectivity_low_current.tif (deflated 9%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>